# Task for Today  

***

## Global Temperature Time Series Forecasting  

Given *time series data about temperatures in different countries*, let's try to predict the **change in average global temperature** for the next 15 years.

We will use a Facebook's Prophet model to make our predictions. 

# Getting Started

In [1]:
import numpy as np
import pandas as pd

import plotly.express as px

#from fbprophet import Prophet

In [2]:
data = pd.read_csv('database.csv', encoding='latin-1')

In [6]:
data

,Area Code,Area,Months Code,Months,Element Code,Element,Unit,Y1961,Y1962,Y1963,...,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019
0,2,Afghanistan,7001,January,7271,Temperature change,°C,0.777,0.062,2.744,...,3.601,1.179,-0.583,1.233,1.755,1.943,3.416,1.201,1.996,2.951
1,2,Afghanistan,7001,January,6078,Standard Deviation,°C,1.950,1.950,1.950,...,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950
2,2,Afghanistan,7002,February,7271,Temperature change,°C,-1.743,2.465,3.919,...,1.212,0.321,-3.201,1.494,-3.187,2.699,2.251,-0.323,2.705,0.086
3,2,Afghanistan,7002,February,6078,Standard Deviation,°C,2.597,2.597,2.597,...,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597
4,2,Afghanistan,7003,March,7271,Temperature change,°C,0.516,1.336,0.403,...,3.390,0.748,-0.527,2.246,-0.076,-0.497,2.296,0.834,4.418,0.234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9651,5873,OECD,7018,JunJulAug,6078,Standard Deviation,°C,0.247,0.247,0.247,...,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247
9652,5873,OECD,7019,SepOctNov,7271,Temperature change,°C,0.036,0.461,0.665,...,0.958,1.106,0.885,1.041,0.999,1.670,1.535,1.194,0.581,1.233
9653,5873,OECD,7019,SepOctNov,6078,Standard Deviation,°C,0.378,0.378,0.378,...,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378,0.378
9654,5873,OECD,7020,Meteorological year,7271,Temperature change,°C,0.165,-0.009,0.134,...,1.246,0.805,1.274,0.991,0.811,1.282,1.850,1.349,1.088,1.297


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9656 entries, 0 to 9655
Data columns (total 66 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Area Code     9656 non-null   int64  
 1   Area          9656 non-null   object 
 2   Months Code   9656 non-null   int64  
 3   Months        9656 non-null   object 
 4   Element Code  9656 non-null   int64  
 5   Element       9656 non-null   object 
 6   Unit          9656 non-null   object 
 7   Y1961         8287 non-null   float64
 8   Y1962         8322 non-null   float64
 9   Y1963         8294 non-null   float64
 10  Y1964         8252 non-null   float64
 11  Y1965         8281 non-null   float64
 12  Y1966         8364 non-null   float64
 13  Y1967         8347 non-null   float64
 14  Y1968         8345 non-null   float64
 15  Y1969         8326 non-null   float64
 16  Y1970         8308 non-null   float64
 17  Y1971         8303 non-null   float64
 18  Y1972         8323 non-null 

# Preprocessing

In [3]:
def preprocess_inputs(df):
    df = df.copy()
    
    # Remove the standard deviation examples
    df = df.query("Element == 'Temperature change'")
    
    # Group countries and take their means
    df = df.groupby('Area').mean()
    
    # Use only time series data
    df = df.loc[:, 'Y1961':]
    
    # Get global mean temperature change
    df = pd.DataFrame(df.mean()).reset_index(drop=False)
    
    # Rename columns
    df.columns=['ds', 'y']
    
    # Fix year column
    df['ds'] = df['ds'].apply(lambda x: x[1:]).astype(np.int)
    
    return df

In [4]:
time_series = preprocess_inputs(data)

C:\Users\aless\AppData\Local\Temp\ipykernel_15736\153721755.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby('Area').mean()
C:\Users\aless\AppData\Local\Temp\ipykernel_15736\153721755.py:20: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df['ds'] = df['ds'].apply(lambda x: x[1:]).astype(np.int)


In [10]:
time_series

,ds,y
0,1961,0.143032
1,1962,-0.028398
2,1963,-0.026297
3,1964,-0.122865
4,1965,-0.224154
5,1966,0.095070
6,1967,-0.131975
7,1968,-0.167841
8,1969,0.105694
9,1970,0.072189


In [5]:
fig = px.line(
    time_series,
    x='ds',
    y='y',
    labels={'ds': "Year", 'y': "Change in temperature (°C)"},
    title="Average Global Temperature Change Over Time"
)

fig.show()

# In-Sample Forecast

In [6]:
time_train = time_series.iloc[:44, :].copy()
time_test = time_series.iloc[44:, :].copy()

In [ ]:
in_model = Prophet()
in_model.fit(time_train)

In [ ]:
in_forecast = in_model.predict(time_test).loc[:, ['ds', 'yhat']]
in_forecast['ds'] = in_forecast['ds'].apply(lambda x: x.year)
in_forecast

In [ ]:
in_result_df = time_series.merge(in_forecast, on='ds', how='left')

fig = px.line(
    in_result_df,
    x='ds',
    y=['y', 'yhat'],
    color_discrete_sequence=['black', 'red'],
    labels={'ds': "Year"},
    title="Average Global Temperature Change Over Time"
)

fig.show()

# Out-of-Sample Forecast

In [ ]:
future_df = pd.DataFrame(np.arange(2020, 2046), columns=['ds'])
future_df

In [ ]:
out_model = Prophet()
out_model.fit(time_series)

In [ ]:
out_forecast = out_model.predict(future_df).loc[:, ['ds', 'yhat']]
out_forecast['ds'] = out_forecast['ds'].apply(lambda x: x.year)
out_forecast

In [ ]:
out_result_df = pd.concat([time_series, out_forecast], axis=0)

fig = px.line(
    out_result_df,
    x='ds',
    y=['y', 'yhat'],
    color_discrete_sequence=['black', 'red'],
    labels={'ds': "Year"},
    title="Average Global Temperature Change Over Time"
)

fig.show()